### Run

In [2]:
from utils import make_gif_from_dicom
from models import UNet

netG_B2A = UNet(1, 1, 32)
ckpts = [
    'mlruns/0/4a677293b79e4991bd675f50cf70f0b7/049.ckpt' # bs4
]
make_gif_from_dicom('gif/00023997','gif/test8.gif', netG_B2A, ckpts, device='cuda:3',pad=0)

100%|██████████| 35/35 [00:07<00:00,  4.41it/s]


### 草稿

In [2]:
from pydicom import dcmread
import pylibjpeg
import matplotlib.animation as animation
from IPython.display import HTML

In [3]:
arr = dcmread('gif/00023997').pixel_array

In [ ]:
# 预览，可跳过
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(arr[i],cmap='gray')] for i in range(0,arr.shape[0],1)]
ani = animation.ArtistAnimation(fig, ims, interval=100, repeat_delay=True, blit=False)

HTML(ani.to_jshtml())

In [ ]:
tsr = torch.from_numpy(arr)/255
tfmc = T.Compose([
    T.Resize(256),
    T.Normalize((0.5,), (0.5,))
])
tsr = tfmc(tsr)

imgs = []
for i in tqdm(range(tsr.shape[0])):
    B = tsr[i].unsqueeze(0).unsqueeze(0)
    fakeA = netG_B2A.model(B)
    fakeA = torchvision.utils.make_grid(fakeA,normalize=True)
#     fakeA = T.functional.adjust_contrast(fakeA,2)
#     fakeA = T.functional.adjust_gamma(fakeA,1.5)
    B = torchvision.utils.make_grid(B,normalize=True)
    
    fakeA = T.ToPILImage()(fakeA)
    fakeA = ImageOps.autocontrast(fakeA,cutoff=1)
    fakeA = T.ToTensor()(fakeA)
    
    B_fakeA = torch.stack((B,fakeA),dim=0)
    a = torchvision.utils.make_grid(B_fakeA)
    img = T.ToPILImage()(a)
    
    imgs.append(img)

In [1158]:
img.save('gif/test7.gif', save_all=True, append_images=imgs)